Examples require an initialized GsSession and relevant entitlements. `run_analytics` scope is required for the functionality covered in this tutorial. External clients need to substitute thier own client id and client secret below. Please refer to <a href="https://developer.gs.com/docs/gsquant/guides/Authentication/2-gs-session/"> Sessions</a> for details.

In [1]:
from gs_quant.session import GsSession
GsSession.use(client_id=None, client_secret=None, scopes=('run_analytics',))

## Measures

A measure is a metric that can be calculated on an instrument, like a dollar price. Below is a table of supported measures and their definitions.


|Measure                                                                                                	                |Definition                                                                                                                                                                                               |
|---------------------------------------------------------------------------------------------------------------------------|---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
|[Dollar Price](https://marquee-dev.web.gs.com/docs/gsquant/api/risk.html#gs_quant.risk.DollarPrice)	                    |Price of the instrument in US Dollars                                                                                                                                                                    |
|[Price](https://marquee-dev.web.gs.com/docs/gsquant/api/risk.html#gs_quant.risk.Price)	                                    |Price of the instrument in the local currency                                                                                                                                                            |
|[IRDelta](https://marquee-dev.web.gs.com/docs/gsquant/api/risk.html#gs_quant.risk.IRDelta)	                                |Change in Dollar Price (USD present value) due to individual 1bp moves in the interest rate instruments used to build the underlying discount curve (bucketed)                                           |
|[IRDeltaParallel](https://marquee-dev.web.gs.com/docs/gsquant/api/risk.html#gs_quant.risk.IRDeltaParallel)	                |Change in Dollar Price (USD present value) due to a parallel 1bp shift in the interest rate instruments used to build the underlying discount curve (scalar)                                             |
|[IRDeltaLocalCcy](https://marquee-dev.web.gs.com/docs/gsquant/api/risk.html#gs_quant.risk.IRDeltaLocalCcy)	                |Change in Price (present value in the denominated currency) due to individual 1bp moves in the interest rate instruments used to build the underlying discount curve (scalar)                            |
|[IRDeltaParallelLocalCcy](https://marquee-dev.web.gs.com/docs/gsquant/api/risk.html#gs_quant.risk.IRDeltaParallelLocalCcy) |Change in Price (present value in the denominated currency) due to a parallel 1bp shift in the interest rate instruments used to build the underlying discount curve (scalar)                            |
|[IRGammaParallel](https://marquee-dev.web.gs.com/docs/gsquant/api/risk.html#gs_quant.risk.IRGammaParallel)	                                |Change in IRDeltaParallel for a parallel 1bp shift in the interest rate instruments used to build the underlying discount curve (scalar)                                                                 |
|[IRVega](https://marquee-dev.web.gs.com/docs/gsquant/api/risk.html#gs_quant.risk.IRVega)	                                |Change in Dollar Price (USD present value) due to individual 1bp moves in the implied volatility (IRAnnualImpliedVol) of instruments used to build the volatility surface (bucketed)                     |
|[IRVegaParallel](https://marquee-dev.web.gs.com/docs/gsquant/api/risk.html#gs_quant.risk.IRVegaParallel)	                |Change in Dollar Price (USD present value) due to a parallel 1bp move in the implied volatility (IRAnnualImpliedVol) of instruments used to build the volatility surface (scalar)                        |
|[IRVegaLocalCcy](https://marquee-dev.web.gs.com/docs/gsquant/api/risk.html#gs_quant.risk.IRVegaLocalCcy)	                |Change in Price (present value in the denominated currency) due to individual 1bp moves in the implied volatility (IRAnnualImpliedVol) of instruments used to build the volatility surface (bucketed)    |
|[IRVegaParallelLocalCcy](https://marquee-dev.web.gs.com/docs/gsquant/api/risk.html#gs_quant.risk.IRVegaParallelLocalCcy)	|Change in Price (present value in the denominated currency) due to a parallel 1bp move in the implied volatility (IRAnnualImpliedVol) of instruments used to build the volatility surface (scalar)       |
|[IRAnnualImpliedVol](https://marquee-dev.web.gs.com/docs/gsquant/api/risk.html#gs_quant.risk.IRAnnualImpliedVol)	        |Interest rate annual implied volatility (in percent)                                                                                                                                                     |
|[IRAnnualATMImpliedVol](https://marquee-dev.web.gs.com/docs/gsquant/api/risk.html#gs_quant.risk.IRAnnualATMImpliedVol)	    |Interest rate annual implied at-the-money volatility (in percent)                                                                                                                                        |
|[IRDailyImpliedVol](https://marquee-dev.web.gs.com/docs/gsquant/api/risk.html#gs_quant.risk.IRDailyImpliedVol)	            |Interest rate daily implied volatility (in basis points)                                                                                                                                                 |
|[IRSpotRate](https://marquee-dev.web.gs.com/docs/gsquant/api/risk.html#gs_quant.risk.IRSpotRate)	                        |Interest rate at-the-money spot rate (in percent)                                                                                                                                                        |
|[IRFwdRate](https://marquee-dev.web.gs.com/docs/gsquant/api/risk.html#gs_quant.risk.IRFwdRate)	                            |Interest rate par rate (in percent)                                                                                                                                                                      |



## Calculating Price Measures

Let's price an instrument. For information on how to define an instrument, please refer to the [Instruments](https://developer.gs.com/docs/gsquant/guides/Pricing-and-Risk/instruments/) guide.

Note, below we resolve the swaption parameters that will be used to price the swaption, thereby mutating the swaption object. If [`resolve()`](https://developer.gs.com/docs/gsquant/api/classes/gs_quant.base.Priceable.html#gs_quant.base.Priceable.resolve), is
not called prior to calling [`price()`](https://developer.gs.com/docs/gsquant/api/classes/gs_quant.base.Priceable.html#gs_quant.base.Priceable.price), the object will be copied and resolved on the fly without mutating the original swaption object.
The preferred behavior may depend on the [`PricingContext`](https://developer.gs.com/docs/gsquant/api/classes/gs_quant.markets.PricingContext.html) - more on this in the [Pricing Context](https://developer.gs.com/docs/gsquant/guides/Pricing-and-Risk/pricing-context/) guide.

[`price()`](https://developer.gs.com/docs/gsquant/api/classes/gs_quant.base.Priceable.html#gs_quant.base.Priceable.price) method will return price in the local currency.


In [ ]:
from gs_quant.instrument import IRSwaption
from gs_quant.common import PayReceive, Currency

swaption = IRSwaption(PayReceive.Receive, '5y', Currency.USD, expiration_date='13m', strike='atm+40', notional_amount=1e8)
swaption.resolve()
swaption.price() # local is USD

All instruments can also priced in dollars.

In [ ]:
swaption.dollar_price() # USD price

## Calculating Risk Measures

We can also calculate risk measures for the defined instrument. Please refer to [the Measures Guide](https://developer.gs.com/docs/gsquant/guides/Pricing-and-Risk/measures/) for the supported risk measures.
Calling [`calc(risk_measure)`](https://developer.gs.com/docs/gsquant/api/classes/gs_quant.base.Priceable.html#gs_quant.base.Priceable.calc) calculates the value of the risk measure and can return a float, a dataframe or a future thereof, depending on how [`PricingContext`](https://developer.gs.com/docs/gsquant/api/classes/gs_quant.markets.PricingContext.html) is being used - more on this in the [Pricing Context guide](https://developer.gs.com/docs/gsquant/guides/Pricing-and-Risk/pricing-context/).

Calculate a scalar value like implied volatility:

In [ ]:
import gs_quant.risk as risk

swaption.calc(risk.IRAnnualImpliedVol) * 10000

Calculate a structured value like vega:

In [ ]:
swaption.calc(risk.IRVega)

See [measures](#Measures) table for information on units.

#### Disclaimer
This website may contain links to websites and the content of third parties ("Third Party Content"). We do not monitor, review or update, and do not have any control over, any Third Party Content or third party websites. We make no representation, warranty or guarantee as to the accuracy, completeness, timeliness or reliability of any Third Party Content and are not responsible for any loss or damage of any sort resulting from the use of, or for any failure of, products or services provided at or from a third party resource. If you use these links and the Third Party Content, you acknowledge that you are doing so entirely at your own risk.